In [ ]:
import sys
sys.path.append("..")
from figutils import *
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches

In [ ]:
def l1(x, y):
    return np.mean(np.abs(x - y))

In [ ]:
scene = "teaser"
methods = ["baseline", "cv_ps"]
method_names = ["Baseline", "Ours"]


it = 19
output_dir = os.path.join(OUTPUT_DIR, scene)

img_ref = mi.TensorXf(mi.Bitmap(os.path.join(output_dir, f"img_ref.exr")))

n_steps = 500

noisy_imgs = []
final_imgs = []
results = []
final_errors = []
for method in methods:
    noisy_img = mi.TensorXf(mi.Bitmap(os.path.join(output_dir, method, "img", f"{it:04d}.exr")))
    img_final = mi.TensorXf(mi.Bitmap(os.path.join(output_dir, method, f"img_final.exr")))
    final_errors.append(l1(img_final, img_ref))
    final_imgs.append(img_final)
    noisy_imgs.append(noisy_img)
    results.append(np.load(os.path.join(output_dir, method, "result.npz")))



In [ ]:
h, w = final_imgs[0].shape[:2]
plot_w = h

inset_wspace = 0.02
inset_hspace = 0.02
# img_ratios = [1, 2*w/h, 1, 1]
inset_aspect = gridspec_aspect(2, 1, 1, 1, wspace=inset_wspace, hspace=inset_hspace)

img_wspace = 0.0
img_aspect = gridspec_aspect(1, 2, w/2, h, wspace=img_wspace)

plot_r = 2.0
outer_ratios = [inset_aspect, img_aspect, inset_aspect, inset_aspect, plot_r/2]
outer_wspace = 0.02
outer_aspect = gridspec_aspect(1, 2, outer_ratios, 1, wspace=outer_wspace)

crop1 = [0.35, 0.22, 0.1]
crop2 = [0.5, 0.52, 0.12]
crops = [crop1, crop2]
crop_colors = ["r", "g"]

fig = plt.figure(1, figsize=(PAGE_WIDTH, PAGE_WIDTH / outer_aspect))
outer = fig.add_gridspec(1, 5, width_ratios=outer_ratios, wspace=outer_wspace)
baseline_gs = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=outer[0], hspace=inset_hspace)
ours_gs = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=outer[2], hspace=inset_hspace)
ref_gs = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=outer[3], hspace=inset_hspace)
img_gs = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer[1], wspace=img_wspace, hspace=0)
plot_gs = gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=outer[4], hspace=0.02)

img_mixed = noisy_imgs[0].numpy()
img_mixed[:, w//2:] = noisy_imgs[1][:, w//2:]
img_mixed[:, w//2-2:w//2+2] = 1.0

main_ax = fig.add_subplot(img_gs[:])
main_ax.imshow(mi.util.convert_to_bitmap(img_mixed), interpolation='none')
disable_ticks(main_ax)
# disable_border(main_ax)

# main_ax.text(2*w/3, 1.0, "CV (ours)", fontsize=DEFAULT_FONTSIZE)
# main_ax.text(0.33, 1.0, "Baseline", fontsize=DEFAULT_FONTSIZE)

inset_gridspecs = [baseline_gs, ours_gs]
# Plot insets
for l, ((rx, ry, s), color) in enumerate(zip(crops, crop_colors)):
    left = int(rx*w)
    right = left + int(s*w)
    top = int(ry*h)
    bottom = top + int(s*w)
    size = int(s*w)
    main_rect = patches.Rectangle((left, top), size, size, linewidth=1.5, edgecolor=color, facecolor='none')
    main_ax.add_patch(main_rect)


    for j, method_name in enumerate(method_names):
        ax = fig.add_subplot(inset_gridspecs[j][l])
        ax.imshow(mi.util.convert_to_bitmap(final_imgs[j][top:bottom, left:right]), interpolation='none')
        disable_ticks(ax)
        # disable_border(ax)
        plt.setp(ax.spines.values(), color=color)
        rect = patches.Rectangle((0, 0), size-1, size-1, linewidth=1.5, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        if l == 0:
            ax.set_title(method_name, y=1.0)

    ax = fig.add_subplot(ref_gs[l])
    ax.imshow(mi.util.convert_to_bitmap(img_ref[top:bottom, left:right]), interpolation='none')
    disable_ticks(ax)
    # disable_border(ax)
    plt.setp(ax.spines.values(), color=color)
    rect = patches.Rectangle((0, 0), size-1, size-1, linewidth=1.5, edgecolor=color, facecolor='none')
    ax.add_patch(rect)

    if l == 0:
        ax.set_title("Reference", y=1.0)


# Plot loss
from matplotlib.ticker import FormatStrFormatter
ax_loss = fig.add_subplot(plot_gs[0])
ax_loss.locator_params(nbins=4, axis='y')
ax_loss.yaxis.set_major_formatter(FormatStrFormatter('%.1e'))
ax_var = fig.add_subplot(plot_gs[1])
for j, method_name in enumerate(method_names):
    loss = results[j]["loss"]
    x = np.linspace(0, n_steps, len(loss))
    ax_loss.semilogy(x, loss, label=method_name)
    var = results[j]["var"]
    ax_var.semilogy(x, var)

ax_loss.set_title(r"$\mathcal{L}^1$ Error", y=1.0)
ax_loss.yaxis.tick_right()
ax_loss.legend()
ax_var.set_title("Variance", y=-0.4)
ax_var.yaxis.tick_right()

save_fig("teaser", pad_inches=0.02)
